In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
from google.colab import files
import matplotlib.pyplot as plt

In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!ls "/content/drive/My Drive/data_x_data"

 baseball_data				'IEOR 135 Athlete Projection System'
 Basketball_Data.gsheet			'Minute Meeting Notes.gdoc'
 batting.ipynb				 nba2019.csv
'Copy of Innovation Navigator.gslides'	 NBA_Analysis.ipynb
'Data Cleaning Directive.gdoc'		 NBA_Cleaning.ipynb
 finalnba.csv				 pitching.ipynb
 finalnba.gsheet


In [0]:
#In order to run all data files in each of our individual laptops, we must make sure the filing system is the same for everyone.
# Make sure your data files are all located in the My Drive and have the same file name as below.
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [0]:
#cleaned nba 2019-2020 season.
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1lO_RW1lRzAVYHO1TVqk2UPT1whCnwORLeQR-zE7LVRw/edit#gid=1145225836')
sheet_one = wb.worksheet('2019-2020 Per Game')
data = sheet_one.get_all_values()
nba2019 = pd.DataFrame(data)
nba2019.columns = nba2019.iloc[0]
nba2019 = nba2019.iloc[1:]
nba2019

,k,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
1,1,Steven Adams\adamsst01,C,26,OKC,58,58,27,4.5,7.6,0.591,0,0.1,0.333,4.5,7.6,0.593,0.593,1.9,3.2,0.59,3.4,6,9.4,2.4,0.9,1.1,1.5,1.9,10.9
2,2,Bam Adebayo\adebaba01,PF,22,MIA,65,65,34.4,6.3,11.1,0.567,0,0.2,0.077,6.3,10.9,0.576,0.568,3.6,5.3,0.69,2.5,8,10.5,5.1,1.2,1.3,2.8,2.5,16.2
3,3,LaMarcus Aldridge\aldrila01,C,34,SAS,53,53,33.1,7.4,15,0.493,1.2,3,0.389,6.2,12,0.519,0.532,3,3.6,0.827,1.9,5.5,7.4,2.4,0.7,1.6,1.4,2.4,18.9
4,4,Nickeil Alexander-Walker\alexani01,SG,21,NOP,41,0,12.2,1.9,5.5,0.339,1,2.9,0.342,0.9,2.7,0.336,0.427,0.4,0.7,0.607,0.2,1.8,2,1.8,0.3,0.2,1,1.1,5.1
5,5,Grayson Allen\allengr01,SG,24,MEM,30,0,16.6,2.6,5.9,0.449,1.1,3,0.363,1.5,2.8,0.541,0.543,1,1.2,0.857,0.2,2,2.2,1.4,0.2,0,0.8,1.2,7.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
620,510,Thaddeus Young\youngth01,PF,31,CHI,64,16,24.9,4.2,9.4,0.448,1.2,3.5,0.356,3,5.9,0.501,0.513,0.7,1.1,0.583,1.5,3.5,4.9,1.8,1.4,0.4,1.6,2.1,10.3
621,511,Trae Young\youngtr01,PG,21,ATL,60,60,35.3,9.1,20.8,0.437,3.4,9.5,0.361,5.7,11.4,0.501,0.519,8,9.3,0.86,0.5,3.7,4.3,9.3,1.1,0.1,4.8,1.7,29.6
622,512,Cody Zeller\zelleco01,C,27,CHO,58,39,23.1,4.3,8.3,0.524,0.3,1.3,0.24,4,7,0.577,0.543,2.1,3.1,0.682,2.8,4.3,7.1,1.5,0.7,0.4,1.3,2.4,11.1
623,513,Ante Žižić\zizican01,C,23,CLE,22,0,10,1.9,3.3,0.569,0,0,,1.9,3.3,0.569,0.569,0.6,0.9,0.737,0.8,2.2,3,0.3,0.3,0.2,0.5,1.2,4.4


In [0]:
#Drop unless/redundant features from stat table.
use_col = ['k','FG','FGA','3P','3PA','2P','2PA', 'FT', 'FTA', 'TRB']
nba2019 = nba2019.drop(columns=use_col)
nba2019.head()

,Player,Pos,Age,Tm,G,GS,MP,FG%,3P%,2P%,eFG%,FT%,ORB,DRB,AST,STL,BLK,TOV,PF,PTS
1,Steven Adams\adamsst01,C,26,OKC,58,58,27,0.591,0.333,0.593,0.593,0.59,3.4,6,2.4,0.9,1.1,1.5,1.9,10.9
2,Bam Adebayo\adebaba01,PF,22,MIA,65,65,34.4,0.567,0.077,0.576,0.568,0.69,2.5,8,5.1,1.2,1.3,2.8,2.5,16.2
3,LaMarcus Aldridge\aldrila01,C,34,SAS,53,53,33.1,0.493,0.389,0.519,0.532,0.827,1.9,5.5,2.4,0.7,1.6,1.4,2.4,18.9
4,Nickeil Alexander-Walker\alexani01,SG,21,NOP,41,0,12.2,0.339,0.342,0.336,0.427,0.607,0.2,1.8,1.8,0.3,0.2,1,1.1,5.1
5,Grayson Allen\allengr01,SG,24,MEM,30,0,16.6,0.449,0.363,0.541,0.543,0.857,0.2,2,1.4,0.2,0,0.8,1.2,7.4


In [0]:
#cleaned nba 2019-2020 season. Need Tm for now to add PER from another dataframe. Remove duplicates that had players traded mid-season (kept only total stat)
nba2019['Player'] = nba2019['Player'].str.replace('\\\\.+', '')
nba2019['StartYear'] = 2019
dup_tot = nba2019[nba2019['Tm'] == 'TOT']
dup = np.array(nba2019[nba2019['Tm'] == 'TOT']['Player'])
nondup = nba2019[~nba2019['Player'].isin(dup)]
nba2019 = pd.concat([nondup, dup_tot])
nba2019 = nba2019.drop(columns= 'Tm')



In [0]:
#get cleaned PER from other table for 2019-2020 season. Will combine with table above (this will be our Y value for our regression prediction model)
sheet_one = wb.worksheet('2019-2020 Advanced')
data = sheet_one.get_all_values()
nba2019per = pd.DataFrame(data)
nba2019per.columns = nba2019per.iloc[0]
nba2019per = nba2019per.iloc[1:]
nba2019per['Player'] = nba2019per['Player'].str.replace('\\\\.+', '')
nba2019per = nba2019per.loc[:, ['Player','Tm','PER']]

#remove duplicates once again.
dup_tot = nba2019per[nba2019per['Tm'] == 'TOT']
dup = np.array(nba2019per[nba2019per['Tm'] == 'TOT']['Player'])
nondup = nba2019per[~nba2019per['Player'].isin(dup)]
nba2019per = pd.concat([nondup, dup_tot])
nba2019per.head()

#Now combine table based on player name
nba2019 = nba2019.merge(nba2019per, on='Player').drop(columns = 'Tm')
nba2019


,Player,Pos,Age,G,GS,MP,FG%,3P%,2P%,eFG%,FT%,ORB,DRB,AST,STL,BLK,TOV,PF,PTS,StartYear,PER
0,Steven Adams,C,26,58,58,27,0.591,0.333,0.593,0.593,0.59,3.4,6,2.4,0.9,1.1,1.5,1.9,10.9,2019,20.8
1,Bam Adebayo,PF,22,65,65,34.4,0.567,0.077,0.576,0.568,0.69,2.5,8,5.1,1.2,1.3,2.8,2.5,16.2,2019,20.6
2,LaMarcus Aldridge,C,34,53,53,33.1,0.493,0.389,0.519,0.532,0.827,1.9,5.5,2.4,0.7,1.6,1.4,2.4,18.9,2019,19.8
3,Nickeil Alexander-Walker,SG,21,41,0,12.2,0.339,0.342,0.336,0.427,0.607,0.2,1.8,1.8,0.3,0.2,1,1.1,5.1,2019,7.6
4,Grayson Allen,SG,24,30,0,16.6,0.449,0.363,0.541,0.543,0.857,0.2,2,1.4,0.2,0,0.8,1.2,7.4,2019,11.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509,Noah Vonleh,C-PF,24,33,1,11,0.562,0.2,0.635,0.579,0.8,1.1,2.6,0.8,0.3,0.2,0.6,1.2,3.8,2019,15.6
510,Derrick Walton,PG,24,26,1,9.5,0.462,0.417,0.533,0.59,0.778,0.1,0.5,1,0.3,0,0.2,0.8,2,2019,8.7
511,Paul Watson,SF-SG,25,5,0,5,0,0,0,0,0.5,0,0.8,1,0.2,0.2,0,0.2,0.2,2019,-0.5
512,Andrew Wiggins,SF-SG,24,54,54,34.4,0.447,0.332,0.506,0.504,0.709,1.2,3.9,3.7,0.8,1,2.4,2.4,21.8,2019,16.4


In [0]:
#cleaned nba 2018-2019 season.
sheet_one = wb.worksheet('2018-2019 Per Game')
data = sheet_one.get_all_values()
nba2018 = pd.DataFrame(data)
nba2018.columns = nba2018.iloc[0]
nba2018 = nba2018.iloc[1:]

#Drop unless/redundant features from stat table.
use_col = ['Rk','FG','FGA','3P','3PA','2P','2PA', 'FT', 'FTA', 'TRB']
nba2018 = nba2018.drop(columns=use_col)
nba2018.head()

#cleaned nba 2018-2019 season. Need Tm for now to add PER from another dataframe. Remove duplicates that had players traded mid-season (kept only total stat)
nba2018['Player'] = nba2018['Player'].str.replace('\\\\.+', '')
nba2018['StartYear'] = 2018
dup_tot = nba2018[nba2018['Tm'] == 'TOT']
dup = np.array(nba2018[nba2018['Tm'] == 'TOT']['Player'])
nondup = nba2018[~nba2018['Player'].isin(dup)]
nba2018 = pd.concat([nondup, dup_tot])
nba2018 = nba2018.drop(columns= 'Tm')

#get cleaned PER from other table for 2018-2019 season. Will combine with table above (this will be our Y value for our regression prediction model)
sheet_one = wb.worksheet('2018-2019 Advanced')
data = sheet_one.get_all_values()
nba2018per = pd.DataFrame(data)
nba2018per.columns = nba2018per.iloc[0]
nba2018per = nba2018per.iloc[1:]
nba2018per['Player'] = nba2018per['Player'].str.replace('\\\\.+', '')
nba2018per = nba2018per.loc[:, ['Player','Tm','PER']]

#remove duplicates once again.
dup_tot = nba2018per[nba2018per['Tm'] == 'TOT']
dup = np.array(nba2018per[nba2018per['Tm'] == 'TOT']['Player'])
nondup = nba2018per[~nba2018per['Player'].isin(dup)]
nba2018per = pd.concat([nondup, dup_tot])
nba2018per.head()

#Now combine table based on player name
nba2018 = nba2018.merge(nba2018per, on='Player').drop(columns = 'Tm')
nba2018

,Player,Pos,Age,G,GS,MP,FG%,3P%,2P%,eFG%,FT%,ORB,DRB,AST,STL,BLK,TOV,PF,PTS,StartYear,PER
0,Álex Abrines,SG,25,31,2,19,0.357,0.323,0.5,0.487,0.923,0.2,1.4,0.6,0.5,0.2,0.5,1.7,5.3,2018,6.3
1,Quincy Acy,PF,28,10,0,12.3,0.222,0.133,0.667,0.278,0.7,0.3,2.2,0.8,0.1,0.4,0.4,2.4,1.7,2018,2.9
2,Jaylen Adams,PG,22,34,1,12.6,0.345,0.338,0.361,0.459,0.778,0.3,1.4,1.9,0.4,0.1,0.8,1.3,3.2,2018,7.6
3,Steven Adams,C,25,80,80,33.4,0.595,0,0.596,0.595,0.5,4.9,4.6,1.6,1.5,1,1.7,2.6,13.9,2018,18.5
4,Bam Adebayo,C,21,82,28,23.3,0.576,0.2,0.588,0.579,0.735,2,5.3,2.2,0.9,0.8,1.5,2.5,8.9,2018,17.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525,Jonas Valančiūnas,C,26,49,27,22.3,0.559,0.292,0.586,0.572,0.795,2.2,6.4,1.4,0.4,1.1,1.8,3,15.6,2018,24.4
526,Christian Wood,PF,23,21,2,12,0.521,0.346,0.571,0.56,0.732,0.8,3.1,0.4,0.3,0.5,0.8,0.8,8.2,2018,21.9
527,Delon Wright,PG,26,75,13,22.7,0.434,0.298,0.492,0.478,0.793,0.9,2.6,3.3,1.2,0.4,1,1.4,8.7,2018,15.5
528,Tyler Zeller,C,29,6,1,15.5,0.533,0,0.552,0.533,0.778,1.8,2.2,0.7,0.2,0.5,0.7,3.3,7.7,2018,17.3


In [0]:
#Now add column for all nba team selections
nba2018['Award'] = 'None'
dictfirst = {'Giannis Antetokounmpo':'First', 'Paul George':'First', 'Nikola Jokić':'First', 'James Harden':'First', 'Stephen Curry':'First',
             'Kevin Durant':'Second', 'Kawhi Leonard':'Second', 'Joel Embiid':'Second', 'Damian Lillard': 'Second', 'Kyrie Irving':'Second',
             'Blake Griffin':'Third', 'LeBron James':'Third', 'Rudy Gobert':'Third', 'Russell Westbrook': 'Third', 'Kemba Walker':'Third'}
nba2018['Award'] = nba2018['Player'].map(dictfirst)
nba2018['Award'].fillna('None', inplace = True)
nba2018


,Player,Pos,Age,G,GS,MP,FG%,3P%,2P%,eFG%,FT%,ORB,DRB,AST,STL,BLK,TOV,PF,PTS,StartYear,PER,Award
0,Álex Abrines,SG,25,31,2,19,0.357,0.323,0.5,0.487,0.923,0.2,1.4,0.6,0.5,0.2,0.5,1.7,5.3,2018,6.3,None
1,Quincy Acy,PF,28,10,0,12.3,0.222,0.133,0.667,0.278,0.7,0.3,2.2,0.8,0.1,0.4,0.4,2.4,1.7,2018,2.9,None
2,Jaylen Adams,PG,22,34,1,12.6,0.345,0.338,0.361,0.459,0.778,0.3,1.4,1.9,0.4,0.1,0.8,1.3,3.2,2018,7.6,None
3,Steven Adams,C,25,80,80,33.4,0.595,0,0.596,0.595,0.5,4.9,4.6,1.6,1.5,1,1.7,2.6,13.9,2018,18.5,None
4,Bam Adebayo,C,21,82,28,23.3,0.576,0.2,0.588,0.579,0.735,2,5.3,2.2,0.9,0.8,1.5,2.5,8.9,2018,17.9,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525,Jonas Valančiūnas,C,26,49,27,22.3,0.559,0.292,0.586,0.572,0.795,2.2,6.4,1.4,0.4,1.1,1.8,3,15.6,2018,24.4,None
526,Christian Wood,PF,23,21,2,12,0.521,0.346,0.571,0.56,0.732,0.8,3.1,0.4,0.3,0.5,0.8,0.8,8.2,2018,21.9,None
527,Delon Wright,PG,26,75,13,22.7,0.434,0.298,0.492,0.478,0.793,0.9,2.6,3.3,1.2,0.4,1,1.4,8.7,2018,15.5,None
528,Tyler Zeller,C,29,6,1,15.5,0.533,0,0.552,0.533,0.778,1.8,2.2,0.7,0.2,0.5,0.7,3.3,7.7,2018,17.3,None


In [0]:
#cleaned nba2017-2018 season.
sheet_one = wb.worksheet('2017-2018 Per Game')
data = sheet_one.get_all_values()
nba2017 = pd.DataFrame(data)
nba2017.columns = nba2017.iloc[0]
nba2017 = nba2017.iloc[1:]

#Drop unless/redundant features from stat table.
use_col = ['Rk','FG','FGA','3P','3PA','2P','2PA', 'FT', 'FTA', 'TRB']
nba2017 = nba2017.drop(columns=use_col)

#cleaned nba 2017-2018 season. Need Tm for now to add PER from another dataframe. Remove duplicates that had players traded mid-season (kept only total stat)
nba2017['Player'] = nba2017['Player'].str.replace('\\\\.+', '')
nba2017['StartYear'] = 2017
dup_tot = nba2017[nba2017['Tm'] == 'TOT']
dup = np.array(nba2017[nba2017['Tm'] == 'TOT']['Player'])
nondup = nba2017[~nba2017['Player'].isin(dup)]
nba2017 = pd.concat([nondup, dup_tot])
nba2017 = nba2017.drop(columns= 'Tm')

#get cleaned PER from other table for 2017-2018 season. Will combine with table above (this will be our Y value for our regression prediction model)
sheet_one = wb.worksheet('2017-2018 Advanced')
data = sheet_one.get_all_values()
nba2017per = pd.DataFrame(data)
nba2017per.columns = nba2017per.iloc[0]
nba2017per = nba2017per.iloc[1:]
nba2017per['Player'] = nba2017per['Player'].str.replace('\\\\.+', '')
nba2017per = nba2017per.loc[:, ['Player','Tm','PER']]

#remove duplicates once again.
dup_tot = nba2017per[nba2017per['Tm'] == 'TOT']
dup = np.array(nba2017per[nba2017per['Tm'] == 'TOT']['Player'])
nondup = nba2017per[~nba2017per['Player'].isin(dup)]
nba2017per = pd.concat([nondup, dup_tot])

#Now combine table based on player name
nba2017 = nba2017.merge(nba2017per, on='Player').drop(columns = 'Tm')
nba2017


,Player,Pos,Age,G,GS,MP,FG%,3P%,2P%,eFG%,FT%,ORB,DRB,AST,STL,BLK,TOV,PF,PTS,StartYear,PER
0,Álex Abrines,SG,24,75,8,15.1,0.395,0.38,0.443,0.54,0.848,0.3,1.2,0.4,0.5,0.1,0.3,1.7,4.7,2017,9
1,Quincy Acy,PF,27,70,8,19.4,0.356,0.349,0.384,0.496,0.817,0.6,3.1,0.8,0.5,0.4,0.9,2.1,5.9,2017,8.2
2,Steven Adams,C,24,76,76,32.7,0.629,0,0.631,0.629,0.559,5.1,4,1.2,1.2,1,1.7,2.8,13.9,2017,20.6
3,Bam Adebayo,C,20,69,19,19.8,0.512,0,0.523,0.512,0.721,1.7,3.8,1.5,0.5,0.6,1,2,6.9,2017,15.7
4,Arron Afflalo,SG,32,53,3,12.9,0.401,0.386,0.413,0.485,0.846,0.1,1.2,0.6,0.1,0.2,0.4,1.1,3.4,2017,5.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,Dwyane Wade,SG,36,67,3,22.9,0.438,0.288,0.468,0.462,0.714,0.7,3,3.4,0.9,0.7,2.1,1.9,11.4,2017,15
536,Briante Weber,PG,25,18,0,13.2,0.442,0.364,0.469,0.488,0.8,0.4,1.5,1.2,1,0.3,0.8,1.6,2.8,2017,10.1
537,Troy Williams,SF,23,21,1,14.6,0.468,0.289,0.563,0.518,0.667,1,2,0.8,1,0.2,0.9,1.6,6.3,2017,15.1
538,Brandan Wright,PF,30,28,1,13.6,0.578,,0.578,0.578,0.636,1.2,2.2,0.5,0.5,0.9,0.3,1.1,5,2017,19


In [0]:
#Now add column for all nba team selections
nba2017['Award'] = 'None'
dictfirst = {'Kevin Durant':'First', 'LeBron James':'First', 'Anthony Davis':'First', 'James Harden':'First', 'Damian Lillard':'First',
             'LaMarcus Aldridge':'Second', 'Giannis Antetokounmpo':'Second', 'Joel Embiid':'Second', 'Russell Westbrook': 'Second', 'DeMar DeRozan':'Second',
             'Stephen Curry':'Third', 'Victor Oladipo':'Third', 'Karl-Anthony Towns':'Third', 'Paul George': 'Third', 'Jimmy Butler':'Third'}
nba2017['Award'] = nba2017['Player'].map(dictfirst)
nba2017['Award'].fillna('None', inplace = True)
nba2017

,Player,Pos,Age,G,GS,MP,FG%,3P%,2P%,eFG%,FT%,ORB,DRB,AST,STL,BLK,TOV,PF,PTS,StartYear,PER,Award
0,Álex Abrines,SG,24,75,8,15.1,0.395,0.38,0.443,0.54,0.848,0.3,1.2,0.4,0.5,0.1,0.3,1.7,4.7,2017,9,None
1,Quincy Acy,PF,27,70,8,19.4,0.356,0.349,0.384,0.496,0.817,0.6,3.1,0.8,0.5,0.4,0.9,2.1,5.9,2017,8.2,None
2,Steven Adams,C,24,76,76,32.7,0.629,0,0.631,0.629,0.559,5.1,4,1.2,1.2,1,1.7,2.8,13.9,2017,20.6,None
3,Bam Adebayo,C,20,69,19,19.8,0.512,0,0.523,0.512,0.721,1.7,3.8,1.5,0.5,0.6,1,2,6.9,2017,15.7,None
4,Arron Afflalo,SG,32,53,3,12.9,0.401,0.386,0.413,0.485,0.846,0.1,1.2,0.6,0.1,0.2,0.4,1.1,3.4,2017,5.8,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,Dwyane Wade,SG,36,67,3,22.9,0.438,0.288,0.468,0.462,0.714,0.7,3,3.4,0.9,0.7,2.1,1.9,11.4,2017,15,None
536,Briante Weber,PG,25,18,0,13.2,0.442,0.364,0.469,0.488,0.8,0.4,1.5,1.2,1,0.3,0.8,1.6,2.8,2017,10.1,None
537,Troy Williams,SF,23,21,1,14.6,0.468,0.289,0.563,0.518,0.667,1,2,0.8,1,0.2,0.9,1.6,6.3,2017,15.1,None
538,Brandan Wright,PF,30,28,1,13.6,0.578,,0.578,0.578,0.636,1.2,2.2,0.5,0.5,0.9,0.3,1.1,5,2017,19,None


In [0]:
#cleaned nba 2016-2017 season
sheet_one = wb.worksheet('2016-2017 Per Game')
data = sheet_one.get_all_values()
nba2016 = pd.DataFrame(data)
nba2016.columns = nba2016.iloc[0]
nba2016 = nba2016.iloc[1:]

#Drop unless/redundant features from stat table.
use_col = ['Rk','FG','FGA','3P','3PA','2P','2PA', 'FT', 'FTA', 'TRB']
nba2016 = nba2016.drop(columns=use_col)

#cleaned nba 2016-2017 season. Need Tm for now to add PER from another dataframe. Remove duplicates that had players traded mid-season (kept only total stat)
nba2016['Player'] = nba2016['Player'].str.replace('\\\\.+', '')
nba2016['StartYear'] = 2016
dup_tot = nba2016[nba2016['Tm'] == 'TOT']
dup = np.array(nba2016[nba2016['Tm'] == 'TOT']['Player'])
nondup = nba2016[~nba2016['Player'].isin(dup)]
nba2016 = pd.concat([nondup, dup_tot])
nba2016 = nba2016.drop(columns= 'Tm')

#get cleaned PER from other table for 2016-2017 season. Will combine with table above (this will be our Y value for our regression prediction model)
sheet_one = wb.worksheet('2016-2017 Advanced')
data = sheet_one.get_all_values()
nba2016per = pd.DataFrame(data)
nba2016per.columns = nba2016per.iloc[0]
nba2016per = nba2016per.iloc[1:]
nba2016per['Player'] = nba2016per['Player'].str.replace('\\\\.+', '')
nba2016per = nba2016per.loc[:, ['Player','Tm','PER']]

#remove duplicates once again.
dup_tot = nba2016per[nba2016per['Tm'] == 'TOT']
dup = np.array(nba2016per[nba2016per['Tm'] == 'TOT']['Player'])
nondup = nba2016per[~nba2016per['Player'].isin(dup)]
nba2016per = pd.concat([nondup, dup_tot])

#Now combine table based on player name
nba2016 = nba2016.merge(nba2016per, on='Player').drop(columns = 'Tm')
nba2016

,Player,Pos,Age,G,GS,MP,FG%,3P%,2P%,eFG%,FT%,ORB,DRB,AST,STL,BLK,TOV,PF,PTS,StartYear,PER
0,Álex Abrines,SG,23,68,6,15.5,0.393,0.381,0.426,0.531,0.898,0.3,1,0.6,0.5,0.1,0.5,1.7,6,2016,10.1
1,Steven Adams,C,23,80,80,29.9,0.571,0,0.572,0.571,0.611,3.5,4.2,1.1,1.1,1,1.8,2.4,11.3,2016,16.5
2,Arron Afflalo,SG,31,61,45,25.9,0.44,0.411,0.457,0.514,0.892,0.1,1.9,1.3,0.3,0.1,0.7,1.7,8.4,2016,8.9
3,Alexis Ajinça,C,28,39,15,15,0.5,0,0.511,0.5,0.725,1.2,3.4,0.3,0.5,0.6,0.8,2,5.3,2016,12.9
4,Cole Aldrich,C,28,62,0,8.6,0.523,,0.523,0.523,0.682,0.8,1.7,0.4,0.4,0.4,0.3,1.4,1.7,2016,12.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481,Briante Weber,PG,24,20,0,10.3,0.417,0.1,0.48,0.425,0.688,0.4,0.9,1.1,0.6,0.1,0.4,0.8,3.1,2016,11
482,Deron Williams,PG,32,64,44,25.9,0.438,0.363,0.486,0.509,0.826,0.2,2.1,5.6,0.5,0.1,2.2,2.2,11,2016,14
483,Derrick Williams,PF,25,50,11,16.1,0.443,0.309,0.531,0.504,0.652,0.4,2.2,0.6,0.3,0.1,0.6,1.2,6.1,2016,10.6
484,Lou Williams,SG,30,81,1,24.6,0.429,0.365,0.48,0.511,0.88,0.3,2.2,3,1,0.2,2,1.1,17.5,2016,21.4


In [0]:
#Now add column for all nba team selections
nba2016['Award'] = 'None'
dictfirst = {'Kawhi Leonard':'First', 'LeBron James':'First', 'Anthony Davis':'First', 'James Harden':'First', 'Russell Westbrook':'First',
             'Kevin Durant':'Second', 'Giannis Antetokounmpo':'Second', 'Rudy Gobert':'Second', 'Stephen Curry': 'Second', 'Isaiah Thomas':'Second',
             'Draymond Green':'Third', 'DeMar DeRozan':'Third', 'DeAndre Jordan':'Third', 'John Wall': 'Third', 'Jimmy Butler':'Third'}
nba2016['Award'] = nba2016['Player'].map(dictfirst)
nba2016['Award'].fillna('None', inplace = True)
nba2016

,Player,Pos,Age,G,GS,MP,FG%,3P%,2P%,eFG%,FT%,ORB,DRB,AST,STL,BLK,TOV,PF,PTS,StartYear,PER,Award
0,Álex Abrines,SG,23,68,6,15.5,0.393,0.381,0.426,0.531,0.898,0.3,1,0.6,0.5,0.1,0.5,1.7,6,2016,10.1,None
1,Steven Adams,C,23,80,80,29.9,0.571,0,0.572,0.571,0.611,3.5,4.2,1.1,1.1,1,1.8,2.4,11.3,2016,16.5,None
2,Arron Afflalo,SG,31,61,45,25.9,0.44,0.411,0.457,0.514,0.892,0.1,1.9,1.3,0.3,0.1,0.7,1.7,8.4,2016,8.9,None
3,Alexis Ajinça,C,28,39,15,15,0.5,0,0.511,0.5,0.725,1.2,3.4,0.3,0.5,0.6,0.8,2,5.3,2016,12.9,None
4,Cole Aldrich,C,28,62,0,8.6,0.523,,0.523,0.523,0.682,0.8,1.7,0.4,0.4,0.4,0.3,1.4,1.7,2016,12.7,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481,Briante Weber,PG,24,20,0,10.3,0.417,0.1,0.48,0.425,0.688,0.4,0.9,1.1,0.6,0.1,0.4,0.8,3.1,2016,11,None
482,Deron Williams,PG,32,64,44,25.9,0.438,0.363,0.486,0.509,0.826,0.2,2.1,5.6,0.5,0.1,2.2,2.2,11,2016,14,None
483,Derrick Williams,PF,25,50,11,16.1,0.443,0.309,0.531,0.504,0.652,0.4,2.2,0.6,0.3,0.1,0.6,1.2,6.1,2016,10.6,None
484,Lou Williams,SG,30,81,1,24.6,0.429,0.365,0.48,0.511,0.88,0.3,2.2,3,1,0.2,2,1.1,17.5,2016,21.4,None


In [0]:
#cleaned nba 2015 - 2016 season.
sheet_one = wb.worksheet('2015-2016 Per Game')
data = sheet_one.get_all_values()
nba2015 = pd.DataFrame(data)
nba2015.columns = nba2015.iloc[0]
nba2015 = nba2015.iloc[1:]

#Drop unless/redundant features from stat table.
use_col = ['Rk','FG','FGA','3P','3PA','2P','2PA', 'FT', 'FTA', 'TRB']
nba2015 = nba2015.drop(columns=use_col)

#cleaned nba 2015-2016 season. Need Tm for now to add PER from another dataframe. Remove duplicates that had players traded mid-season (kept only total stat)
nba2015['Player'] = nba2015['Player'].str.replace('\\\\.+', '')
nba2015['StartYear'] = 2015
dup_tot = nba2015[nba2015['Tm'] == 'TOT']
dup = np.array(nba2015[nba2015['Tm'] == 'TOT']['Player'])
nondup = nba2015[~nba2015['Player'].isin(dup)]
nba2015 = pd.concat([nondup, dup_tot])
nba2015 = nba2015.drop(columns= 'Tm')

#get cleaned PER from other table for 2015-2016 season. Will combine with table above (this will be our Y value for our regression prediction model)
sheet_one = wb.worksheet('2015-2016 Advanced')
data = sheet_one.get_all_values()
nba2015per = pd.DataFrame(data)
nba2015per.columns = nba2015per.iloc[0]
nba2015per = nba2015per.iloc[1:]
nba2015per['Player'] = nba2015per['Player'].str.replace('\\\\.+', '')
nba2015per = nba2015per.loc[:, ['Player','Tm','PER']]

#remove duplicates once again.
dup_tot = nba2015per[nba2015per['Tm'] == 'TOT']
dup = np.array(nba2015per[nba2015per['Tm'] == 'TOT']['Player'])
nondup = nba2015per[~nba2015per['Player'].isin(dup)]
nba2015per = pd.concat([nondup, dup_tot])

#Now combine table based on player name
nba2015 = nba2015.merge(nba2015per, on='Player').drop(columns = 'Tm')
nba2015

,Player,Pos,Age,G,GS,MP,FG%,3P%,2P%,eFG%,FT%,ORB,DRB,AST,STL,BLK,TOV,PF,PTS,StartYear,PER
0,Quincy Acy,PF,25,59,29,14.8,0.556,0.388,0.606,0.6,0.735,1.1,2.1,0.5,0.5,0.4,0.5,1.7,5.2,2015,14.7
1,Jordan Adams,SG,21,2,0,7.5,0.333,0,0.4,0.333,0.6,0,1,1.5,1.5,0,1,1,3.5,2015,17.3
2,Steven Adams,C,22,80,80,25.2,0.613,,0.613,0.613,0.582,2.7,3.9,0.8,0.5,1.1,1.1,2.8,8,2015,15.5
3,Arron Afflalo,SG,30,71,57,33.4,0.443,0.382,0.469,0.5,0.84,0.3,3.4,2,0.4,0.1,1.2,2,12.8,2015,10.9
4,Alexis Ajinça,C,27,59,17,14.6,0.476,0,0.478,0.476,0.839,1.3,3.3,0.5,0.3,0.6,0.9,2.3,6,2015,13.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,Marcus Thornton,SG,28,61,8,18.2,0.399,0.337,0.46,0.482,0.848,0.6,1.8,1.4,0.8,0.1,0.8,1.5,9.7,2015,14.7
472,Beno Udrih,PG,33,44,5,16.1,0.434,0.341,0.461,0.472,0.9,0.2,1.5,2.6,0.3,0,1.2,1.2,4.7,2015,9.5
473,Anderson Varejão,C,33,53,0,9.4,0.427,0,0.431,0.427,0.64,0.7,2,0.7,0.3,0.2,0.4,1.3,2.6,2015,11.1
474,Briante Weber,PG,23,7,4,24.1,0.359,0,0.412,0.359,0.75,1.1,2.4,3,1.3,0.4,1.3,1.7,4.4,2015,8.9


In [0]:
#Now add column for all nba team selections
nba2015['Award'] = 'None'
dictfirst = {'Kawhi Leonard':'First', 'LeBron James':'First', 'DeAndre Jordan':'First', 'James Harden':'First', 'Stephen Curry':'First',
             'Kevin Durant':'Second', 'Draymond Green':'Second', 'Damian Lillard':'Second', 'Chris Paul': 'Second', 'DeMarcus Cousins':'Second',
             'Paul George':'Third', 'LaMarcus Aldridge':'Third', 'Andre Drummond':'Third', 'Klay Thompson': 'Third', 'Kyle Lowry':'Third'}
nba2015['Award'] = nba2015['Player'].map(dictfirst)
nba2015['Award'].fillna('None', inplace = True)
nba2015

,Player,Pos,Age,G,GS,MP,FG%,3P%,2P%,eFG%,FT%,ORB,DRB,AST,STL,BLK,TOV,PF,PTS,StartYear,PER,Award
0,Quincy Acy,PF,25,59,29,14.8,0.556,0.388,0.606,0.6,0.735,1.1,2.1,0.5,0.5,0.4,0.5,1.7,5.2,2015,14.7,None
1,Jordan Adams,SG,21,2,0,7.5,0.333,0,0.4,0.333,0.6,0,1,1.5,1.5,0,1,1,3.5,2015,17.3,None
2,Steven Adams,C,22,80,80,25.2,0.613,,0.613,0.613,0.582,2.7,3.9,0.8,0.5,1.1,1.1,2.8,8,2015,15.5,None
3,Arron Afflalo,SG,30,71,57,33.4,0.443,0.382,0.469,0.5,0.84,0.3,3.4,2,0.4,0.1,1.2,2,12.8,2015,10.9,None
4,Alexis Ajinça,C,27,59,17,14.6,0.476,0,0.478,0.476,0.839,1.3,3.3,0.5,0.3,0.6,0.9,2.3,6,2015,13.8,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,Marcus Thornton,SG,28,61,8,18.2,0.399,0.337,0.46,0.482,0.848,0.6,1.8,1.4,0.8,0.1,0.8,1.5,9.7,2015,14.7,None
472,Beno Udrih,PG,33,44,5,16.1,0.434,0.341,0.461,0.472,0.9,0.2,1.5,2.6,0.3,0,1.2,1.2,4.7,2015,9.5,None
473,Anderson Varejão,C,33,53,0,9.4,0.427,0,0.431,0.427,0.64,0.7,2,0.7,0.3,0.2,0.4,1.3,2.6,2015,11.1,None
474,Briante Weber,PG,23,7,4,24.1,0.359,0,0.412,0.359,0.75,1.1,2.4,3,1.3,0.4,1.3,1.7,4.4,2015,8.9,None


In [0]:
#Now combine all tables except for the 2019-2020 season together for our train-test split analysis. We will use 2019-2020 season for our 
#finalized prediction and compare with current media predictions today.
finalnba = pd.concat([nba2015,nba2016, nba2017,nba2018])
finalnba

,Player,Pos,Age,G,GS,MP,FG%,3P%,2P%,eFG%,FT%,ORB,DRB,AST,STL,BLK,TOV,PF,PTS,StartYear,PER,Award
0,Quincy Acy,PF,25,59,29,14.8,0.556,0.388,0.606,0.6,0.735,1.1,2.1,0.5,0.5,0.4,0.5,1.7,5.2,2015,14.7,None
1,Jordan Adams,SG,21,2,0,7.5,0.333,0,0.4,0.333,0.6,0,1,1.5,1.5,0,1,1,3.5,2015,17.3,None
2,Steven Adams,C,22,80,80,25.2,0.613,,0.613,0.613,0.582,2.7,3.9,0.8,0.5,1.1,1.1,2.8,8,2015,15.5,None
3,Arron Afflalo,SG,30,71,57,33.4,0.443,0.382,0.469,0.5,0.84,0.3,3.4,2,0.4,0.1,1.2,2,12.8,2015,10.9,None
4,Alexis Ajinça,C,27,59,17,14.6,0.476,0,0.478,0.476,0.839,1.3,3.3,0.5,0.3,0.6,0.9,2.3,6,2015,13.8,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525,Jonas Valančiūnas,C,26,49,27,22.3,0.559,0.292,0.586,0.572,0.795,2.2,6.4,1.4,0.4,1.1,1.8,3,15.6,2018,24.4,None
526,Christian Wood,PF,23,21,2,12,0.521,0.346,0.571,0.56,0.732,0.8,3.1,0.4,0.3,0.5,0.8,0.8,8.2,2018,21.9,None
527,Delon Wright,PG,26,75,13,22.7,0.434,0.298,0.492,0.478,0.793,0.9,2.6,3.3,1.2,0.4,1,1.4,8.7,2018,15.5,None
528,Tyler Zeller,C,29,6,1,15.5,0.533,0,0.552,0.533,0.778,1.8,2.2,0.7,0.2,0.5,0.7,3.3,7.7,2018,17.3,None


In [0]:
#2019-2020 season data for reference
nba2019

,Player,Pos,Age,G,GS,MP,FG%,3P%,2P%,eFG%,FT%,ORB,DRB,AST,STL,BLK,TOV,PF,PTS,StartYear,PER
0,Steven Adams,C,26,58,58,27,0.591,0.333,0.593,0.593,0.59,3.4,6,2.4,0.9,1.1,1.5,1.9,10.9,2019,20.8
1,Bam Adebayo,PF,22,65,65,34.4,0.567,0.077,0.576,0.568,0.69,2.5,8,5.1,1.2,1.3,2.8,2.5,16.2,2019,20.6
2,LaMarcus Aldridge,C,34,53,53,33.1,0.493,0.389,0.519,0.532,0.827,1.9,5.5,2.4,0.7,1.6,1.4,2.4,18.9,2019,19.8
3,Nickeil Alexander-Walker,SG,21,41,0,12.2,0.339,0.342,0.336,0.427,0.607,0.2,1.8,1.8,0.3,0.2,1,1.1,5.1,2019,7.6
4,Grayson Allen,SG,24,30,0,16.6,0.449,0.363,0.541,0.543,0.857,0.2,2,1.4,0.2,0,0.8,1.2,7.4,2019,11.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509,Noah Vonleh,C-PF,24,33,1,11,0.562,0.2,0.635,0.579,0.8,1.1,2.6,0.8,0.3,0.2,0.6,1.2,3.8,2019,15.6
510,Derrick Walton,PG,24,26,1,9.5,0.462,0.417,0.533,0.59,0.778,0.1,0.5,1,0.3,0,0.2,0.8,2,2019,8.7
511,Paul Watson,SF-SG,25,5,0,5,0,0,0,0,0.5,0,0.8,1,0.2,0.2,0,0.2,0.2,2019,-0.5
512,Andrew Wiggins,SF-SG,24,54,54,34.4,0.447,0.332,0.506,0.504,0.709,1.2,3.9,3.7,0.8,1,2.4,2.4,21.8,2019,16.4


In [0]:
from google.colab import files
finalnba.to_csv('finalnba.csv') 
files.download('finalnba.csv')

In [0]:
nba2019.to_csv('nba2019.csv') 
files.download('nba2019.csv')